In [1]:
from qaravan.tensorQ import *
from qaravan.core import *

In [3]:
# testing environments 
n = 4
sv = random_sv(n)

gate_list = [H(0), CNOT([1,0], n), CNOT([2,1], n), CNOT([3,2], n)]
circ = Circuit(gate_list, n)
sim = StatevectorSim(circ)
final_state = sim.run(progress_bar=False).reshape(2**n)
overlap_a = sv.conj().T @ final_state

for gate_idx in range(len(circ)):
    env, mat = sv_environment(circ, sv, gate_idx)
    overlap_b = np.trace(env @ mat)
    print(np.allclose(overlap_a, overlap_b))

True
True
True
True


In [4]:
# testing environments from cached states 

def test_fast_environment():
    target_sv = random_sv(4)
    circ = two_local_circ([(0,1), (2,3), (1,2), (0,1), (2,3), (1,2)])

    for gate_idx in range(len(circ)):
        env_slow, _ = sv_environment(circ, target_sv, gate_idx)
        pre_states, post_states = cache_states(circ, target_sv)
        env_fast = partial_overlap(pre_states[gate_idx], post_states[gate_idx], skip=circ[gate_idx].indices)

        assert np.allclose(env_fast, env_slow), f"Mismatch in env at gate {gate_idx}"

test_fast_environment()

Pre states: 100%|██████████| 6/6 [00:00<00:00, 3004.87it/s]
Post states: 6it [00:00, 6084.58it/s]
Pre states: 100%|██████████| 6/6 [00:00<00:00, 835.88it/s]
Post states: 6it [00:00, 2992.72it/s]
Pre states: 100%|██████████| 6/6 [00:00<00:00, 3024.37it/s]
Post states: 6it [00:00, 6141.00it/s]
Pre states: 100%|██████████| 6/6 [00:00<00:00, 5929.74it/s]
Post states: 6it [00:00, 3003.80it/s]
Pre states: 100%|██████████| 6/6 [00:00<00:00, 5557.82it/s]
Post states: 6it [00:00, ?it/s]
Pre states: 100%|██████████| 6/6 [00:00<00:00, 3007.75it/s]
Post states: 6it [00:00, 5996.15it/s]


In [5]:
# testing environment_update()

def test_environment_update():
    target_sv = random_sv(4)
    circ = two_local_circ([(0,1), (2,3), (1,2), (0,1), (2,3), (1,2)])
    pre_states, post_states = cache_states(circ, target_sv)

    simple_costs, update_costs, cache_costs = [], [], []
    for idx in range(len(circ)-1):
        update_costs.append(environment_update(circ, idx, pre_states, post_states, direction='right'))

        sim = StatevectorSim(circ)
        ansatz = sim.run(progress_bar=False).reshape(2**n)
        simple_costs.append(1 - np.abs(target_sv.conj().T @ ansatz))

        pre_state = pre_states[idx+1]
        post_state = post_states[idx+1]
        env = partial_overlap(pre_state, post_state, skip=circ[idx+1].indices)
        cache_costs.append(1 - np.abs(np.trace(env @ circ[idx+1].matrix)))

    for idx in reversed(range(1, len(circ))):
        update_costs.append(environment_update(circ, idx, pre_states, post_states, direction='left'))
    
        sim = StatevectorSim(circ)
        ansatz = sim.run(progress_bar=False).reshape(2**n)
        simple_costs.append(1 - np.abs(target_sv.conj().T @ ansatz))

        pre_state = pre_states[idx-1]
        post_state = post_states[idx-1]
        env = partial_overlap(pre_state, post_state, skip=circ[idx-1].indices)
        cache_costs.append(1 - np.abs(np.trace(env @ circ[idx-1].matrix)))

    assert np.allclose(simple_costs, update_costs), "Mismatch in update costs"
    assert np.allclose(simple_costs, cache_costs), "Mismatch in cache costs"

test_environment_update()

Pre states: 100%|██████████| 6/6 [00:00<00:00, 3551.99it/s]
Post states: 6it [00:00, ?it/s]


In [6]:
# testing environment based state prep
skeleton = [(0,1), (2,3), (1,2), (0,1), (2,3)]
target_sv = random_sv(4)

context = RunContext(
    progress_interval=15,
    max_iter=50,
    stop_ratio=1e-8,
    checkpoint_file="test.pickle", 
    checkpoint_interval=40,
    )
    
circ, cost_list = environment_state_prep(target_sv, skeleton=skeleton, context=context)

with open('test.pickle', 'rb') as f: 
    opt_state = pickle.load(f)

opt_state['step'], opt_state['cost_list'][::40]

Pre states: 100%|██████████| 5/5 [00:00<00:00, 2483.60it/s]
Post states: 5it [00:00, 5012.31it/s]

Step 0 at time 2025-04-16T12:56:20: cost = 0.006529893102142026
[Checkpoint saved at step 0]
Step 15 at time 2025-04-16T12:56:20: cost = 0.00047650836974089117
Step 30 at time 2025-04-16T12:56:20: cost = 0.00037368990055308693


[Checkpoint saved at step 40]
Step 45 at time 2025-04-16T12:56:21: cost = 0.0002824618369370313
Max iterations reached with cost 0.0002627376710409024


(40,
 [np.float64(0.8423496243383297),
  np.float64(0.0005417134776106725),
  np.float64(0.000513536276964377),
  np.float64(0.0004829707908580172),
  np.float64(0.000449782671134602),
  np.float64(0.0004151753734987773),
  np.float64(0.0003805116885934323),
  np.float64(0.000347087718263972),
  np.float64(0.00031593584926625784)])

In [7]:
# resuming from previous run
context = RunContext(
    progress_interval=15,
    max_iter=100,
    stop_ratio=1e-8,
    checkpoint_file="test.pickle", 
    checkpoint_interval=40,
    resume=True
    )
    
circ, cost_list = environment_state_prep(target_sv, skeleton=skeleton, context=context)

with open('test.pickle', 'rb') as f: 
    opt_state = pickle.load(f)

opt_state['step'], opt_state['cost_list'][::40]

[Resuming from checkpoint: step 40]
[Checkpoint saved at step 40]
Step 45 at time 2025-04-16T12:56:33: cost = 0.00027733405742547124
Step 60 at time 2025-04-16T12:56:33: cost = 0.00021554562841674674
Step 75 at time 2025-04-16T12:56:33: cost = 0.00017678854199021377
[Checkpoint saved at step 80]
Step 90 at time 2025-04-16T12:56:33: cost = 0.00015244896031663124
Max iterations reached with cost 0.00014209936521569766


(80,
 [np.float64(0.8423496243383297),
  np.float64(0.0005417134776106725),
  np.float64(0.000513536276964377),
  np.float64(0.0004829707908580172),
  np.float64(0.000449782671134602),
  np.float64(0.0004151753734987773),
  np.float64(0.0003805116885934323),
  np.float64(0.000347087718263972),
  np.float64(0.00031593584926625784),
  np.float64(0.00028772062218251904),
  np.float64(0.0002627376710409024),
  np.float64(0.0002409842638103621),
  np.float64(0.00022225714979473565),
  np.float64(0.00020624395485258074),
  np.float64(0.00019259214710243278),
  np.float64(0.00018095334783763128),
  np.float64(0.00017100758672583538)])